In [1]:
import pandas as pd
import networkx as nx

In [12]:
df = pd.read_csv('./dataset/netflix_titles.csv')

In [13]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [14]:
df['directors'] = df['director'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(',')])

In [15]:
df['actors'] = df['cast'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(',')])

In [16]:
df['categories'] = df['listed_in'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(',')])

In [17]:
df['countries'] = df['country'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(',')])

In [18]:
df = df[['title', 'directors', 'actors', 'categories', 'countries']]

In [23]:
gn = nx.Graph(label='Netflix')


In [24]:
for i, row in df.iterrows():
    title = row['title']
    gn.add_node(title, label='MOVIE')
    for actor in row['actors']:
        gn.add_node(actor, label='PERSON')
        gn.add_edge(title, actor, label='ACTED_IN')
    
    for director in row['directors']:
        gn.add_node(director, label='PERSON')
        gn.add_edge(title, director, label='DIRECTED_BY')
    
    for cat in row['categories']:
        gn.add_node(cat, label='CATEGORY')
        gn.add_edge(title, cat, label='CATEGORY_IN')
    
    for country in row['countries']:
        gn.add_node(country, label='COUNTRY')
        gn.add_edge(title, country, label='COUNTRY_IN')

        

In [26]:
from node2vec import Node2Vec
n2v = Node2Vec(gn, dimensions=100, walk_length=16, num_walks=10)
model = n2v.fit(window=5, min_count=1)

Generating walks (CPU: 1): 100%|██████████| 10/10 [04:44<00:00, 30.37s/it]


In [31]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
node_embeddings = model.wv.vectors
node_embeddings_2d = tsne.fit_transform(node_embeddings)

In [30]:
import matplotlib.pyplot as plt
node_ids = model.wv.index2word
node_labels = [gn.nodes[node_id]['label'] for node_id in node_ids]
label_map = {
    "MOVIE": 'blue',
    "PERSON": 'red',
    "CATEGORY": 'green',
    "COUNTRY": 'yellow'
}

node_colors = [label_map[label] for label in node_labels]
plt.figure()
plt.scatter(node_embeddings_2d[:, 0], node_embeddings_2d[:, 1], c=node_colors, cmap='jet')


NameError: name 'node_embeddings_2d' is not defined

<Figure size 432x288 with 0 Axes>